# An architecture for achieving greatness
1. [Containers](#containers)
    - [VM vs. Container](#VM-vs-Container)
    - [Software packaging](#Software-packaging)
    - [Building and deploying](#Building-and-deploying)
    - [Sharing is caring - or not](#Sharing-is-caring---or-not)
    - [How does it scale](#How-does-it-scale)
2. [DC/OS](#dc/os)
    - [One OS to rule them all](#One-OS-to-rule-them-all)
        - [Mesos](#Mesos)
            - [The minimal cluster resource manager](#The-minimal-cluster-resource-manager)
            - [User space and system space - just like linux](#User-space-and-system-space---just-like-linux)
        - [Marathon - the worlds oldest scheduler](#Marathon---the-worlds-oldest-scheduler)
        - [And a bunch of other things](#And-a-bunch-of-other-things)
    - [DC/OS vs. Kubernetes vs. Swarm](#DC/OS-vs.-Kubernetes-vs.-Swarm)
    - [Creating apps](#creating-apps)
        - [An example app - Memcached](#An-example-app---Memcached)
3. [Azure](#Azure)
    - [Deploying DC/OS](#deploying-dc/os)
    - [Deployment and monitoring](#deployment-and-monitoring)

In [ ]:
# Download some images...
download("https://dcos.io/docs/1.9/img/dcos-components-1.9.png", "dcos-arch.png");
download("https://www.dropbox.com/s/b0txdyqkklddj9u/ContainerOnHost.png?dl=0", "containeronhost.png");
download("https://www.dropbox.com/s/4qvnwi0h8c044ma/VMOnHost.png?dl=0", "vmonhost.png");

# .. and load some packages
using FileIO

In [ ]:
dcosarch = load("dcos-arch.png")
conthost = load("containeronhost.png")
vmhost = load("vmonhost.png");

## Containers

### VM vs. Container

<tr>
    <td> <img src="containeronhost.png" alt="Drawing" style="width: 400px;"/> </td>
    <td> <img src="vmonhost.png" alt="Drawing" style="width: 400px;"/> </td>
    </tr>

### Software packaging

##### A basic Julia script checking some metrics against Marathon

In [ ]:
using Requests
qry = "http://172.16.0.5:8080/v2/apps/nordicdatasummit/tasks"
while true
        curl = get(qry)
        println(Requests.json(curl))
        sleep(20)
end

##### A basic Dockerfile for the script

```bash
FROM centos:7

# INSTALL JULIA  
ENV JULIA_TAG="0.6"
ENV JULIA_BIN="https://julialang-s3.julialang.org/bin/linux/x64/${JULIA_TAG}/julia-${JULIA_TAG}-latest-linux-x86_64.tar.gz"
RUN yum --enablerepo=extras install -y epel-release && \
    yum update -y && \
    curl ${JULIA_BIN} | tar zxf - -C /tmp && \
    cp -r /tmp/julia*/* /usr/local && \
    rm -rf /tmp/*

# Update packages and install Requests
RUN julia -e 'Pkg.update()' && \
    yum install -y http-parser unzip && \
    julia -e 'Pkg.add("Requests")' && \
    julia -e 'using Requests'

# Add example julia script to be executed on start
ADD example.jl /
ENTRYPOINT ["julia", "/example.jl"]
```

### Building and deploying

### Sharing is caring - or not

### How does it scale

## DC/OS

### One OS to rule them all

<tr>
    <td> <img src="dcos-arch.png" alt="Drawing" style="width: 500px;"/> </td>
</tr>

### Mesos

#### User space and system space - just like linux

<tr>
    <td> <img src="https://dcos.io/docs/1.9/img/dcos-node-types.png" alt="Drawing" style="width: 500px;"/> </td>
</tr>


### Marathon

### DC/OS vs. Kubernetes vs. Swarm

### Creating apps

##### Deploying the example Docker image on DC/OS with Marathon

```json
{
  "id": "nordicdatasummit",
  "cpus": 0.5,
  "mem": 1000,
  "container": {
    "type": "DOCKER",
    "docker": {
      "image": "laketide/nordicdata",
      "network": "BRIDGE",
      "forcePullImage": true
    }
  }
}
```

#### Deploying a Universe app to Marathon

##### A Memcached json definition example

```json
{
  "id": "/memcached",
  "cmd": "memcached -m 1024 -vv",
  "instances": 1,
  "cpus": 1,
  "mem": 2000,
  "disk": 0,
  "gpus": 0,
  "constraints": [],
  "maxLaunchDelaySeconds": 3600,
  "container": {
    "type": "DOCKER",
    "volumes": [],
    "docker": {
      "image": "memcached",
      "network": "BRIDGE",
      "portMappings": [
        {
          "containerPort": 11211,
          "hostPort": 11211,
          "servicePort": 0,
          "protocol": "tcp",
          "name": "connect",
          "labels": {
            "VIP_0": "/memcached:11211"
          }
        }
      ],
      "privileged": true,
      "forcePullImage": false
    }
  },
  "healthChecks": [
    {
      "gracePeriodSeconds": 600,
      "intervalSeconds": 10,
      "timeoutSeconds": 10,
      "maxConsecutiveFailures": 3,
      "portIndex": 0,
      "protocol": "TCP"
    }
  ],
  "upgradeStrategy": {
    "minimumHealthCapacity": 0,
    "maximumOverCapacity": 0
  },
  "unreachableStrategy": {
    "inactiveAfterSeconds": 300,
    "expungeAfterSeconds": 600
  },
  "killSelection": "YOUNGEST_FIRST",
  "requirePorts": true
}
```

## Azure

### Deploying DC/OS

### Deployment and monitoring